In [ ]:
import pandas as pd

from summer.utils import ref_times_to_dti

from autumn.core import inputs
from autumn.core.utils.display import pretty_print
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline
model = p.run_baseline_model(baseline_params)
derived_df = model.get_derived_outputs_df()
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [ ]:
# pretty_print(baseline_params)

In [ ]:
def get_comparison_df(indicator):
    return pd.DataFrame({
        "modelled": derived_df[indicator],
        "data": ts_set_dates[indicator]
    })

In [ ]:
get_comparison_df("notifications").plot()

In [ ]:
get_comparison_df("hospital_admissions").plot()

In [ ]:
get_comparison_df("icu_admissions").plot()

In [ ]:
get_comparison_df("infection_deaths").plot()

In [ ]:
sc_models = p.run_scenario_models(
    model, p.param_set.scenarios,
)

In [ ]:
indicator = "notifications"
sc_1_results = sc_models[1].get_derived_outputs_df()
pd.DataFrame(
    {
        0: derived_df[indicator],
        1: sc_1_results[indicator],
    }
).plot()

In [ ]:
# Check population distribution by vaccination status
sc_1_results[[f"prop_immune_{strat}" for strat in model._stratifications[3].strata]].plot.area()